## 1. 랭체인 프롬프트 작성(MessagesPlaceholder)

In [14]:
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

load_dotenv()
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash"
)

prompt = ChatPromptTemplate([
    {
        "role":"user",
        "content":[
            {
                "type":"text",
                "text": "이 이미지에 대한 보고서를 한글로 작성하세요"
            },
            {
                "type":"image",
                "source_type":"base64",
                "mime_type":"image/png",
                "data":"{image_data}"
            }
        ]
    },
    MessagesPlaceholder(variable_name="messages")
    
])

import base64
img_path = r"C:\Users\Sese\Pictures\Screenshots\스크린샷 2025-06-07 195837.png"
img = open(img_path, "rb").read()
image_data = base64.b64encode(img).decode('utf-8')

messages = [
    {
        "role":"user",
        "content":
            {
                "type":"text",
                "text":"보고서의 형식은 다음과 같습니다:\n\n1. 제목\n2. 요약\n3. 주요 내용\n4. 결론"
            }
    }
]

chain = prompt | llm
response = chain.invoke({"image_data":image_data, "messages":[]})
print(response.content)

## 이미지 보고서

**개요:**

이미지는 파이썬 코드를 보여줍니다. 코드 스니펫은 두 개의 가상 챗봇(앱등이와 삼엽충) 간의 논쟁을 시뮬레이션하는 데 사용되는 것으로 보입니다. 논쟁의 주제는 삼성과 애플 제품의 장단점입니다. 코드는 `langchain` 라이브러리를 사용하여 챗봇을 구축하고 OpenAI의 GPT-4o 모델을 사용하는 것으로 보입니다.

**코드 내용 분석:**

*   **변수 정의:** `bot2_reply = response2.content`와 `current_message = bot2_reply`는 챗봇의 응답을 저장하는 변수를 정의합니다.
*   **출력:** `print(f"{bot2_name}: {bot2_reply}")`는 챗봇의 이름과 응답을 콘솔에 출력합니다.
*   **챗봇 대화:** 이미지 중앙 부분은 실제로 챗봇 간의 대화 내용을 보여줍니다.  각 챗봇은 삼성과 애플 제품에 대한 극단적인 의견을 제시하며, 비속어와 공격적인 표현을 사용합니다.
*   **라이브러리 임포트:** `import re`, `from langchain.schema import HumanMessage, SystemMessage`, `from langchain.chat_models import ChatOpenAI`는 코드에서 사용되는 라이브러리를 임포트합니다.
*   **DebateBot 클래스:** `class DebateBot`는 챗봇 논쟁을 관리하는 데 사용되는 클래스를 정의합니다. `__init__` 메서드는 챗봇 모델과 온도(temperature)를 초기화합니다.

**대화 내용 요약:**

대화는 다음과 같은 특징을 보입니다.

*   **극단적인 주장:** 각 챗봇은 자신의 선호 브랜드에 대해 극단적인 주장을 펼칩니다.
*   **비속어 사용:** 대화에 비속어가 빈번하게 등장합니다.
*   **인신 공격:** 상대방을 비난하는 인신 공격적인 발언이 나타납니다.
*   **주요 논쟁점:** 삼성의 조잡함, 애플의 폐쇄성 및 감성 마케

## 2. Tool 호출

In [27]:
from langchain_experimental.tools import PythonREPLTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

load_dotenv()


llm = ChatOpenAI(model="gpt-4o")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

python_tool = PythonREPLTool()

print(python_tool.invoke("print(100+200)"))

def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)

prompt = ChatPromptTemplate.from_messages([
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": """You are Raymond Hetting, an expert Python programmer.
Generate only clean, executable Python code that runs correctly in an interactive REPL environment (no '__main__', no file-level guards).
Write short, elegant, and PEP8-compliant code.
Return only the code — no explanations, no markdown, no formatting."""
            }
        ]
    },
    {
        "role": "user",
        "content": "{input}"
    }
])


chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)
print(chain.invoke("로또 번호 생성기를 출력하는 코드를 작성하세요."))


300

CODE:
```python
import random

def generate_lotto_numbers():
  """Generates a set of 6 unique random numbers between 1 and 45 (inclusive)."""
  return sorted(random.sample(range(1, 46), 6))

print(generate_lotto_numbers())
```
[2, 10, 11, 15, 22, 40]



In [35]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
prompt = PromptTemplate.from_template(
    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

chain = prompt | llm | StrOutputParser()

image_prompt = chain.invoke(
    {"image_desc":"스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"}
)

print(image_prompt)

Create a neo-classical painting that satirizes people looking at their smartphones. The scene should depict a grand, classical setting reminiscent of famous neoclassical artworks, with ornate columns and dramatic lighting. In the foreground, a diverse group of individuals—men and women of various ages and ethnicities—are engrossed in their smartphones, their expressions reflecting a mixture of fascination and detachment. In the background, traditional neoclassical figures such as philosophers and scholars are engaged in deep discussion, highlighting the contrast between their intellectual pursuits and the modern obsession with technology. The color palette should be rich and warm, with detailed textures, capturing the essence of neoclassicism while merging it with contemporary themes. Include subtle humorous elements, such as a statue looking disapprovingly at the scene, emphasizing the irony of the situation.


In [29]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image

dalle = DallEAPIWrapper(model="dall-e-3", size="1024x1024", quality="standard", n=1)

query = image_prompt
image_url = dalle.run(chain.invoke({"image_desc":query}))

Image(url=image_url, width=500)

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [6]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableSequence
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import Image

dalle = DallEAPIWrapper(model="dall-e-3", size="1024x1024", quality="standard", n=1)


llm = ChatOpenAI(model="gpt-4o-mini")
generate_image_url = RunnableLambda(lambda prompt: dalle.run(prompt))
prompt = PromptTemplate.from_template(
    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

chain = prompt | llm | StrOutputParser() | generate_image_url

image_url = chain.invoke({"image_desc":\
    "복귀자(감귤)와 미복귀자가 서로 다투는 4컷 만화"})
Image(url=image_url, width=500)


## 3. 사용자 정의 도구(Custom Tool)

In [31]:
import re
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool


# 도구를 정의합니다.
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b


@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"


tools = [get_word_length, add_function, naver_news_crawl]

gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
gemini_with_tools = gemini.bind_tools(tools)

response = gemini_with_tools.invoke("What is the length of the word 'teddynote'?")
print(response.tool_calls)
print(response.content)

[{'name': 'get_word_length', 'args': {'word': 'teddynote'}, 'id': 'e74fa9cf-9a95-42ad-b8ca-3771fa7a8f8e', 'type': 'tool_call'}]



In [33]:
response.__dict__

{'content': '',
 'additional_kwargs': {'function_call': {'name': 'get_word_length',
   'arguments': '{"word": "teddynote"}'}},
 'response_metadata': {'prompt_feedback': {'block_reason': 0,
   'safety_ratings': []},
  'finish_reason': 'STOP',
  'model_name': 'gemini-2.0-flash',
  'safety_ratings': []},
 'type': 'ai',
 'name': None,
 'id': 'run--c90fca06-7793-45b0-8a73-1b85eecba3e9-0',
 'example': False,
 'tool_calls': [{'name': 'get_word_length',
   'args': {'word': 'teddynote'},
   'id': 'e74fa9cf-9a95-42ad-b8ca-3771fa7a8f8e',
   'type': 'tool_call'}],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 74,
  'output_tokens': 9,
  'total_tokens': 83,
  'input_token_details': {'cache_read': 0}}}

In [34]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """
    # 도구 호출 결과 리스트를 순회합니다.
    for tool_call_result in tool_call_results:
        # 도구의 이름과 인자를 추출합니다.
        tool_name = tool_call_result["type"]
        tool_args = tool_call_result["args"]

        # 도구 이름과 일치하는 도구를 찾아 실행합니다.
        # next() 함수를 사용하여 일치하는 첫 번째 도구를 찾습니다.
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        if matching_tool:
            # 일치하는 도구를 찾았다면 해당 도구를 실행합니다.
            result = matching_tool.invoke(tool_args)
            # 실행 결과를 출력합니다.
            print(f"[실행도구] {tool_name}\n[실행결과] {result}")
        else:
            # 일치하는 도구를 찾지 못했다면 경고 메시지를 출력합니다.
            print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")


from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# 도구 바인딩 + 도구 파서
chain = gemini_with_tools | JsonOutputToolsParser(tools=tools)

# 실행 결과
tool_call_results = chain.invoke("What is the length of the word 'teddynote'?")

execute_tool_calls(tool_call_results)


[실행도구] get_word_length
[실행결과] 9
